Data cleaning to relational tables

In [ ]:
sql = """create table minister
(
    id                integer primary key,
    "wiki link"       TEXT,
    start             TEXT,
    district_link     TEXT,
    "group"           TEXT,
    member            TEXT,
    party             TEXT,
    district          TEXT,
    is_senator        INT,
    is_representative INT,
    graduated INT
);

insert into minister("wiki link", start, district_link, "group", member, party, district, is_senator,
                            is_representative, graduated)
select distinct "wiki link",
       start,
       district_link,
       "group",
       member,
       party,
       district,
       is_senator,
       is_representative,
       graduated,
       mp_id
from cleaning_data_ministers;

ALTER TABLE minister ADD COLUMN mp_id integer;
UPDATE minister
SET mp_id = a.mp_id from (SELECT distinct member, mp_id from ministers) a WHERE a.member = minister.member;


create TABLE minister as SELECT distinct "wiki link", start, district_link, "group", member, party, district, is_senator, is_representative, graduated from ministers;
ALTER TABLE minister ADD COLUMN mp_id integer;
UPDATE minister
SET mp_id = a.mp_id from (SELECT distinct member, mp_id from ministers) a WHERE a.member = minister.member;

create TABLE minister_education as SELECT distinct ministers.education_id, minister.id as minister_id  from ministers join minister on ministers.member = minister.member;

CREATE TABLE education_acara as SELECT e.fid as education_id,  "ACARA SML ID" as acara_id from acara__education JOIN education e on acara__education.school_name = e.school_name;

drop table  if exists education_acara_dg_tmp;
create table education_acara_dg_tmp
(
    education_id INT not null
        constraint education_acara_education_fid_fk
            references education,
    acara_id     integer not null
        constraint education_acara_acara_fid_fk
            references acara_school_locations_2022,
    id           integer primary key /*autoincrement needs PK*/
);

insert into education_acara_dg_tmp(education_id, acara_id, id)
select education_id, asl.fid as acara_id, id
from education_acara JOIN acara_school_locations_2022 asl on asl."ACARA SML ID" = education_acara.acara_id;

drop table education_acara;

alter table education_acara_dg_tmp
    rename to education_acara;

alter table ministers rename to  cleaning_data_ministers;

drop table  if exists education_acara_dg_tmp;
create table education_acara_dg_tmp
(
    education_id INT not null
        constraint education_acara_education_fid_fk
            references education,
    acara_id     integer not null
        constraint education_acara_acara_fid_fk
            references acara_school_locations_2022,
    id           integer primary key /*autoincrement needs PK*/
);

insert into education_acara_dg_tmp(education_id, acara_id)
select e.fid as education_id, asl.fid as acara_id
from acara__education ae
    JOIN acara_school_locations_2022 asl on asl."ACARA SML ID" = ae."ACARA SML ID"
    JOIN education e  on e.school_name = ae.school_name;
;

drop table education_acara;

alter table education_acara_dg_tmp
    rename to education_acara;

"""


Add some nice views for analysis



In [ ]:
# add mary doyle manually
"""INSERT INTO minister_education(minister_id, education_id)
SELECT id,fid  from minister,  education
where member = 'Mary Doyle' AND  school_name IN  ('Bendigo Senior Secondary College', 'St Joseph’s College,  Echuca' );

INSERT INTO education_acara(education_id, acara_id)
SELECT e.fid, a.fid from education e , acara_school_locations_2022 a
where school_name IN  ('Bendigo Senior Secondary College', 'St Joseph’s College, Echuca' )
AND ( "School Name" LIKE '%joseph%' AND "Suburb" = 'ECHUCA' OR "School Name" = 'Bendigo Senior Secondary College')"""

In [ ]:
import pathlib
import sqlite3

import pandas as pd

from utils import get_dob_gender_from_wikidata

root_data_dir = pathlib.Path("..").resolve() / "data"
ext_data_dir = root_data_dir / "external"

gpkg = root_data_dir / "aped.gpkg"
db_con = sqlite3.connect(gpkg)
ministers = pd.read_sql("SELECT * from minister", db_con)

df = ministers["wiki link"].apply(get_dob_gender_from_wikidata)
ministers["dob"] = pd.to_datetime(df["dob"])
ministers["gender_ident"] = df["genderLabel"]
minister_demogs = ministers[["id", "member", "gender_ident", "dob"]]
minister_demogs.rename(columns={"id": "minister_id"}, inplace=True)
minister_demogs.to_sql(db_con=db_con, name="minister_demogs", if_exists="replace")

In [ ]:
"""
ALTER TABLE minister ADD COLUMN high_school varchar(15);
-- Add school sector use partition over member to check if all values are the same
WITH school_agg AS (
SELECT
    minister.id,
    member,
    array_agg("school sector") as school_sector
FROM minister
    LEFT JOIN minister_education me ON minister.id = me.minister_id
    JOIN education e ON me.education_id = e.id
    JOIN education_acara ea ON e.id = ea.education_id
    JOIN acara_school_locations_2022 asl ON ea.acara_id = asl."acara sml id"
GROUP BY member, minister.id),
    comb as (
SELECT id,
       member,
    CASE
        WHEN 'Government' = ANY(school_sector) AND ('Independent' = ANY(school_sector) OR 'Catholic' = ANY(school_sector))
            THEN 'Both'
        WHEN 'Government' != ANY(school_sector) AND 'Independent'  != ANY(school_sector) AND 'Catholic'  != ANY(school_sector)
            THEN 'Unknown'
        WHEN 'Government' = ANY(school_sector)
            THEN 'Public'
        WHEN 'Independent' = ANY(school_sector)
            THEN 'Non-government'
        WHEN 'Catholic' = ANY(school_sector)
            THEN 'Non-government'
    END AS school_sector_group
FROM school_agg)
UPDATE minister
SET high_school = comb.school_sector_group
FROM comb
WHERE minister.id = comb.id;
"""

In [ ]:
# add house
"""ALTER TABLE minister ADD COLUMN house varchar(6);
UPDATE minister SET house = 'house' WHERE is_representative = True;
UPDATE minister SET house = 'senate' WHERE is_senator = True;
"""

In [ ]:
# Fix members secondary school
"UPDATE public.members_secondary_school SET secondaryschool = 'Rosny College'::text WHERE id = 79::bigint AND phid LIKE '008CH' ESCAPE '#' AND secondaryschool LIKE 'Rosny College (Hobart)' ESCAPE '#' AND ctid = '(0,72)'"
"UPDATE public.members_secondary_school SET secondaryschool = 'St Ignatius College'::text WHERE id = 303::bigint AND phid LIKE '275424' ESCAPE '#' AND secondaryschool LIKE 'St. Ignatius College' ESCAPE '#' AND ctid = '(2,42)'"
"UPDATE public.members_secondary_school SET secondaryschool = 'Lauriston Girls'' School'::text WHERE id = 203::bigint AND phid LIKE '124514' ESCAPE '#' AND secondaryschool LIKE 'Lauriston Girls School' ESCAPE '#' AND ctid = '(1,67)'"

In [ ]:
# Tidy wikipedia members
"""
alter table members_wiki
    rename column item to wikidata_entity;

alter table members_wiki
    rename column start to service_start;

alter table members_wiki
    rename column "end" to service_end;

alter table members_wiki
    rename column district to district_link;

alter table members_wiki
    rename column "itemLabel" to member;

alter table members_wiki
    rename column "districtLabel" to division;

alter table members_wiki
    rename column edu to school_link;

alter table members_wiki
    rename column "eduLabel" to school_name;

alter table members_wiki
    rename column "group" to party_link;

alter table members_wiki
    rename column "groupLabel" to party_name;

"""

In [ ]:
"""
UPDATE education
SET school_link = NULL
WHERE school_link = 'http://www.wikidata.org/entity/None';

UPDATE members
SET wiki_link = NULL
WHERE wiki_link = '';

-- update school link where empty string to null
UPDATE education
SET school_link = NULL
WHERE school_link = '';
"""

In [ ]:
""" INSERT INTO members_education(education_id, member_id)  VALUES
(3,62),
(57,240),
(246,170),
(17,199),
(247,78),
(145,19),
(251,219);"""